<a href="https://colab.research.google.com/github/Apekshaa2908/Enhancing-E-Commerce-Agility-With-Advanced-ETL-Pipeline/blob/main/Streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit
!pip install boto3

In [ ]:
%%writefile app.py
import streamlit as st
import boto3
import pandas as pd
from io import BytesIO

# Streamlit app title
st.title('Upload Order and Return CSV Files to S3 Buckets')

# Input fields for AWS credentials
st.subheader("AWS Credentials")
aws_access_key_id = st.text_input("AWS Access Key ID", type="password")
aws_secret_access_key = st.text_input("AWS Secret Access Key", type="password")

# Define bucket names
ORDER_BUCKET = 'order-apekshaa'
RETURN_BUCKET = 'return-apekshaa'

# Initialize the S3 client only if credentials are provided
if aws_access_key_id and aws_secret_access_key:
    s3 = boto3.client(
        's3',
        aws_access_key_id=aws_access_key_id,
        aws_secret_access_key=aws_secret_access_key,
        region_name='ap-southeast-2'
    )

    # Function to upload files to S3
    def upload_to_s3(file, bucket_name, file_name):
        try:
            # Convert the file to a byte stream
            file_buffer = BytesIO()
            file.to_csv(file_buffer, index=False)
            file_buffer.seek(0)

            # Upload to S3
            s3.put_object(Bucket=bucket_name, Key=file_name, Body=file_buffer.getvalue())
            st.success(f'File {file_name} uploaded to {bucket_name}')
        except Exception as e:
            st.error(f"Error uploading {file_name}: {str(e)}")

    # Upload order_table
    order_file = st.file_uploader("Upload order_table CSV", type=["csv"], key="order_file")
    if order_file:
        order_df = pd.read_csv(order_file)
        st.dataframe(order_df)  # Display uploaded CSV in the app
        if st.button("Upload order_table to S3"):
            upload_to_s3(order_df, ORDER_BUCKET, 'order_table.csv')

    # Upload return_table
    return_file = st.file_uploader("Upload return_table CSV", type=["csv"], key="return_file")
    if return_file:
        return_df = pd.read_csv(return_file)
        st.dataframe(return_df)  # Display uploaded CSV in the app
        if st.button("Upload return_table to S3"):
            upload_to_s3(return_df, RETURN_BUCKET, 'return_table.csv')
else:
    st.warning("Please enter your AWS credentials to proceed.")


In [ ]:
! wget -q -O - ipv4.icanhazip.com
! streamlit run app.py & npx localtunnel --port 8501